<a href="https://colab.research.google.com/github/ezzywd78/MS_AI_Engineer_Course/blob/main/Walmart_Sales_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Retail Analysis with Walmart Data
# Course-end Project 1 — reproducible script
#
# Save this file next to Walmart_Store_sales.csv and run:
#   pip install pandas numpy matplotlib seaborn scikit-learn joblib
#   python retail_analysis_walmart.py
#
# The script:
# - computes the requested basic statistics
# - produces monthly / semester aggregates
# - maps the holiday weeks to holiday names and compares their sales vs non-holiday mean
# - fits simple prediction models for Store 1 and reports RMSE/MAE, selecting the best model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

sns.set(style="whitegrid")
RANDOM_STATE = 42

# ---- 1. Load and prepare data ----
df = pd.read_csv("Walmart_Store_sales.csv")
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")  # file dates are in d-m-Y
df = df.sort_values(['Store', 'Date']).reset_index(drop=True)

# Additional time columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.to_period('Q')
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# ---- 2. Basic statistics tasks ----

# 2.1 Which store has maximum sales (total across period)?
store_total = df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False)
store_with_max_sales = store_total.idxmax()
print("Store with maximum total sales:", store_with_max_sales)
print("Total sales for that store: ${:,.2f}".format(store_total.loc[store_with_max_sales]))

# 2.2 Which store has maximum standard deviation i.e., sales vary a lot.
store_std = df.groupby('Store')['Weekly_Sales'].std().sort_values(ascending=False)
store_with_max_std = store_std.idxmax()
store_mean = df.groupby('Store')['Weekly_Sales'].mean()
coef_var = (store_std / store_mean).sort_values(ascending=False)  # coefficient of variation = std/mean
store_with_max_cv = coef_var.idxmax()
print("\nStore with maximum standard deviation:", store_with_max_std)
print("Standard deviation: ${:,.2f}".format(store_std.loc[store_with_max_std]))
print("Store with maximum coefficient of variation (std/mean):".format(store_with_max_cv))
print("Coefficient of variation: {:.4f}".format(coef_var.loc[store_with_max_cv]))

# 2.3 Which store(s) has good quarterly growth rate in Q3 2012
# We'll calculate two kinds of growth:
#  - QoQ: (Q3_2012 - Q2_2012) / Q2_2012
#  - YoY: (Q3_2012 - Q3_2011) / Q3_2011 (year-over-year)
q_sales = df.copy()
q_sales['YearQuarter'] = q_sales['Date'].dt.to_period('Q')
q_agg = q_sales.groupby(['Store', 'YearQuarter'])['Weekly_Sales'].sum().unstack(fill_value=np.nan)

q3_2012 = q_agg.get('2012Q3')  # Period label format "YYYYQn"
q2_2012 = q_agg.get('2012Q2')
q3_2011 = q_agg.get('2011Q3')

# compute growth safely (avoid division by zero)
qoQ = ((q3_2012 - q2_2012) / q2_2012).replace([np.inf, -np.inf], np.nan)
yoY = ((q3_2012 - q3_2011) / q3_2011).replace([np.inf, -np.inf], np.nan)

# pick stores with positive growth (both metrics positive) and rank them:
growth_df = pd.DataFrame({
    'Q3_2012': q3_2012,
    'Q2_2012': q2_2012,
    'Q3_2011': q3_2011,
    'QoQ_growth_Q3_2012_vs_Q2_2012': qoQ,
    'YoY_growth_Q3_2012_vs_Q3_2011': yoY
}).sort_values('QoQ_growth_Q3_2012_vs_Q2_2012', ascending=False)

print("\nTop 10 stores by QoQ growth (Q3 2012 vs Q2 2012):")
print(growth_df[['Q3_2012','Q2_2012','QoQ_growth_Q3_2012_vs_Q2_2012']].head(10))

# stores with clearly positive QoQ growth:
stores_positive_qoq = growth_df[growth_df['QoQ_growth_Q3_2012_vs_Q2_2012'] > 0].index.tolist()
print("\nStores with positive QoQ growth into Q3 2012 (vs Q2 2012):", stores_positive_qoq)

# 2.4 Holidays with higher sales than mean non-holiday sales
# Define holiday dates mapping (from problem statement) and filter only dates present in data
holiday_map = {
    'Super Bowl': ['2010-02-12', '2011-02-11', '2012-02-10'],
    'Labour Day': ['2010-09-10', '2011-09-09', '2012-09-07'],
    'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23'],
    'Christmas': ['2010-12-31', '2011-12-30']  # 2012-12-28 is outside dataset end maybe
}
# normalize map to datetimes that exist in the dataset
holiday_rows = []
for hname, dates in holiday_map.items():
    for d in dates:
        try:
            dt = pd.to_datetime(d, format="%Y-%m-%d")
            if dt in set(df['Date']):
                holiday_rows.append((hname, dt))
        except Exception:
            continue

# mean sales for non-holiday weeks (all stores together)
non_holiday_mean = df[df['Holiday_Flag'] == 0]['Weekly_Sales'].mean()
print("\nMean weekly sales across all stores during non-holiday weeks: ${:,.2f}".format(non_holiday_mean))

# compute avg total sales across all stores for each holiday date
holiday_stats = []
for hname, dt in holiday_rows:
    # sum across all stores for that week
    total_sales = df[df['Date'] == dt]['Weekly_Sales'].sum()
    holiday_stats.append({'Holiday': hname, 'Date': dt, 'Total_Sales': total_sales})

holiday_stats_df = pd.DataFrame(holiday_stats)
holiday_stats_df['HigherThanNonHolidayMean'] = holiday_stats_df['Total_Sales'] > non_holiday_mean * df['Store'].nunique()
# Note: non_holiday_mean is per-store-week; to compare to total across all stores multiply by number of stores.
print("\nHoliday weeks: total sales across stores and whether they exceed non-holiday average (all stores combined):")
print(holiday_stats_df[['Holiday','Date','Total_Sales','HigherThanNonHolidayMean']])

# 2.5 Monthly and semester view of sales in units and insights
# Monthly totals (all stores combined)
monthly = df.groupby([df['Date'].dt.to_period('M')])['Weekly_Sales'].sum().rename('Total_Sales').to_frame()
monthly.index = monthly.index.to_timestamp()
print("\nMonthly totals (first 12 rows):")
print(monthly.head(12))

# Semester view: define H1 = Jan-Jun, H2 = Jul-Dec per year
df['Semester'] = np.where(df['Month'] <= 6, 'H1', 'H2')
semester_totals = df.groupby(['Year','Semester'])['Weekly_Sales'].sum().unstack().fillna(0)
print("\nSemester totals by year (H1 / H2):")
print(semester_totals)

# Plotting monthly and semester (save figures)
plt.figure(figsize=(12,5))
monthly.plot(kind='line', legend=False, title='Total monthly sales (all stores)')
plt.ylabel('Total Weekly Sales (sum across stores)')
plt.tight_layout()
plt.savefig("monthly_total_sales.png", dpi=150)
plt.close()

plt.figure(figsize=(8,4))
semester_totals.plot(kind='bar', title='Semester sales by Year (H1 vs H2)')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.savefig("semester_sales_by_year.png", dpi=150)
plt.close()

print("\nSaved plots: monthly_total_sales.png and semester_sales_by_year.png")

# ---- 3. Statistical Model for Store 1 ----
# We'll prepare data, create a day-index, basic features, and compare Linear Regression vs RandomForest.
store1 = df[df['Store'] == 1].copy().sort_values('Date').reset_index(drop=True)

# create a numeric date index: days since first date
start_date = store1['Date'].min()
store1['day_index'] = (store1['Date'] - start_date).dt.days

# features: day_index, CPI, Unemployment, Fuel_Price, Temperature, Holiday_Flag, Month (cyclical)
store1['month_sin'] = np.sin(2 * np.pi * store1['Month'] / 12)
store1['month_cos'] = np.cos(2 * np.pi * store1['Month'] / 12)

features = ['day_index', 'CPI', 'Unemployment', 'Fuel_Price', 'Temperature', 'Holiday_Flag', 'month_sin', 'month_cos']
X = store1[features].ffill().bfill()  # Use ffill() and bfill() directly
y = store1['Weekly_Sales']

# Time-aware train/test split: last 12 weeks as test
test_weeks = 12
X_train, X_test = X[:-test_weeks], X[-test_weeks:]
y_train, y_test = y[:-test_weeks], y[-test_weeks:]

# Model 1: Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)

# Model 2: Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print("\nStore 1 prediction results (last {} weeks as test):".format(test_weeks))
print("Linear Regression  RMSE: ${:,.2f}, MAE: ${:,.2f}".format(rmse_lr, mae_lr))
print("Random Forest     RMSE: ${:,.2f}, MAE: ${:,.2f}".format(rmse_rf, mae_rf))

# Choose best model
best_model_name = "LinearRegression" if rmse_lr < rmse_rf else "RandomForest"
print("Selected best model based on RMSE:", best_model_name)

# show linear regression coefficients (if chosen)
coef_df = pd.DataFrame({'feature': features, 'coef': lr.coef_}).sort_values('coef', key=lambda s: s.abs(), ascending=False)
print("\nLinear Regression coefficients (by absolute magnitude):")
print(coef_df)

# Feature importances from RF
fi = pd.DataFrame({'feature': features, 'importance': rf.feature_importances_}).sort_values('importance', ascending=False)
print("\nRandom Forest feature importances:")
print(fi)

# Save predictions to CSV for inspection
predictions = store1[['Date', 'Weekly_Sales']].copy().iloc[-test_weeks:].reset_index(drop=True)
predictions['LR_Pred'] = y_pred_lr
predictions['RF_Pred'] = y_pred_rf
predictions.to_csv("store1_predictions_last_{}weeks.csv".format(test_weeks), index=False)
print("\nSaved predictions to store1_predictions_last_{}weeks.csv".format(test_weeks))

# Plot predictions vs actual for store 1 (last test_weeks)
plt.figure(figsize=(10,5))
plt.plot(predictions['Date'], predictions['Weekly_Sales'], label='Actual', marker='o')
plt.plot(predictions['Date'], predictions['LR_Pred'], label='LinearRegression', marker='x')
plt.plot(predictions['Date'], predictions['RF_Pred'], label='RandomForest', marker='s')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Weekly Sales')
plt.title('Store 1: actual vs predicted (last {} weeks)'.format(test_weeks))
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig("store1_predictions_comparison.png", dpi=150)
plt.close()
print("Saved plot store1_predictions_comparison.png")

# ---- 4. Quick notes / insights (automated suggestions) ----
insights = """
Automated insights summary:
- Store with maximum total sales: {} (see store_total series).
- Store with maximum variability (std): {}.
- Several stores showed positive QoQ growth into Q3 2012 (list printed).
- Holiday weeks comparison: see holiday_stats_df printed above; those holiday weeks with totals above
  the aggregate non-holiday mean (scaled to number of stores) are flagged.
- Monthly and semester aggregates saved as PNGs. Inspect monthly_total_sales.png for seasonality,
  and semester_sales_by_year.png for H1/H2 comparisons.
- Modeling (Store 1): compared a temporal linear model vs RandomForest (features: day index, CPI,
  Unemployment, Fuel_Price, Temperature, Holiday_Flag, cyclical month). The script reports RMSE/MAE.
  RandomForest often captures non-linearity and interactions; LinearRegression is interpretable (coeffs printed).
""".format(store_with_max_sales, store_with_max_std)

with open("insights_summary.txt","w") as f:
    f.write(insights)

print("\nWrote automated insights to insights_summary.txt")

# ---- 5. Recommendations & next steps ----
recommendations = """
Recommended next steps (for better accuracy and business value):
1) Add lag features and rolling means (e.g., sales lag-1, lag-2, 4-week rolling mean) to capture temporal autocorrelation.
2) Use time-series-specific models: SARIMAX (with exogenous variables), FB Prophet (now called 'prophet'),
   or gradient boosting (XGBoost/LightGBM) with time features + lags.
3) Engineer markdown/promotion features: the dataset lacks explicit markdown amounts — try to infer markdown events
   from sudden sales uplift patterns or introduce retailer markup calendars if available.
4) Cross-validate using rolling (time-series) CV instead of a single holdout; tune hyperparameters for tree-based models.
5) Evaluate business metric: weight holiday-week errors more (per problem statement) — e.g., multiply errors on
   the four major holiday weeks by 5 to reflect competition scoring.
6) For supply chain decisions, produce prediction intervals (quantiles) instead of point estimates.
"""

with open("next_steps.txt","w") as f:
    f.write(recommendations)

print("\nWrote recommended next steps to next_steps.txt")


Store with maximum total sales: 20
Total sales for that store: $301,397,792.46

Store with maximum standard deviation: 14
Standard deviation: $317,569.95
Store with maximum coefficient of variation (std/mean):
Coefficient of variation: 0.2297

Top 10 stores by QoQ growth (Q3 2012 vs Q2 2012):
           Q3_2012      Q2_2012  QoQ_growth_Q3_2012_vs_Q2_2012
Store                                                         
7       8262787.39   7290859.27                       0.133308
16      7121541.64   6564335.98                       0.084884
35     11322421.12  10838313.00                       0.044666
26     13675691.91  13155335.57                       0.039555
39     20715116.23  20214128.46                       0.024784
41     18093844.01  17659942.73                       0.024570
44      4411251.16   4306405.78                       0.024346
24     17976377.72  17684218.91                       0.016521
40     12873195.37  12727737.53                       0.011428
23     186414

<Figure size 1200x500 with 0 Axes>

<Figure size 800x400 with 0 Axes>

In [ ]:
from google.colab import files

files.download('retail_analysis_report.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

files.download('retail_analysis_report.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=e1e4d652c1e4e7caab6a495799dd30fd028fdad2414eab082dd5ca8ca49aa58c
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [ ]:
from fpdf import FPDF

def create_report_pdf(output_filename="retail_analysis_report.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Title
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "Walmart Store Sales Retail Analysis Report", 0, 1, "C")
    pdf.ln(10)

    # Add Insights Summary
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Insights Summary", 0, 1, "L") # Renamed from "Automated Insights Summary"
    pdf.set_font("Arial", "", 10)
    try:
        with open("insights_summary.txt", "r") as f:
            insights_text = f.read()
        # Replace problematic character for latin-1 encoding
        insights_text = insights_text.replace('—', '--')
        pdf.multi_cell(0, 5, insights_text)
    except FileNotFoundError:
        pdf.multi_cell(0, 5, "Insights summary file not found.")
    pdf.ln(5)

    # Removed 'Add Next Steps' section
    # pdf.set_font("Arial", "B", 12)
    # pdf.cell(0, 10, "Recommended Next Steps", 0, 1, "L")
    # pdf.set_font("Arial", "", 10)
    # try:
    #     with open("next_steps.txt", "r") as f:
    #         next_steps_text = f.read()
    #     # Replace problematic character for latin-1 encoding
    #     next_steps_text = next_steps_text.replace('—', '--')
    #     pdf.multi_cell(0, 5, next_steps_text)
    # except FileNotFoundError:
    #     pdf.multi_cell(0, 5, "Next steps file not found.")
    # pdf.ln(5)

    # Add Monthly Total Sales Plot
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Monthly Total Sales", 0, 1, "C")
    try:
        pdf.image("monthly_total_sales.png", x=10, y=pdf.get_y(), w=190)
        pdf.set_y(pdf.get_y() + 100) # Adjust y position after image
    except FileNotFoundError:
        pdf.multi_cell(0, 5, "Monthly sales plot not found.")
    pdf.ln(10)

    # Add Semester Sales by Year Plot
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Semester Sales by Year", 0, 1, "C")
    try:
        pdf.image("semester_sales_by_year.png", x=10, y=pdf.get_y(), w=190)
        pdf.set_y(pdf.get_y() + 100) # Adjust y position after image
    except FileNotFoundError:
        pdf.multi_cell(0, 5, "Semester sales plot not found.")
    pdf.ln(10)

    # Add Store 1 Predictions Comparison Plot
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Store 1: Actual vs Predicted Sales", 0, 1, "C")
    try:
        pdf.image("store1_predictions_comparison.png", x=10, y=pdf.get_y(), w=190)
        pdf.set_y(pdf.get_y() + 100) # Adjust y position after image
    except FileNotFoundError:
        pdf.multi_cell(0, 5, "Store 1 predictions plot not found.")
    pdf.ln(10)

    # Add Python Code
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Python Analysis Script", 0, 1, "C")
    pdf.ln(5)
    pdf.set_font("Courier", "", 8) # Use a monospace font for code

    # Get the content of the main analysis script cell directly
    # The cell ID 'zCrMO0jXrgfd' is assumed to contain the analysis script
    # This avoids issues with embedding multi-line strings within multi-line strings
    # For this to work, the agent must have access to the notebook_state to retrieve cell content.
    # In a real execution environment, you might read from a file or a global variable.
    # For the purposes of this interaction, I will manually provide the content from the previous steps.
    code_content = '''
# Retail Analysis with Walmart Data
# Course-end Project 1 — reproducible script
#
# Save this file next to Walmart_Store_sales.csv and run:
#   pip install pandas numpy matplotlib seaborn scikit-learn joblib
#   python retail_analysis_walmart.py
#
# The script:
# - computes the requested basic statistics
# - produces monthly / semester aggregates
# - maps the holiday weeks to holiday names and compares their sales vs non-holiday mean
# - fits simple prediction models for Store 1 and reports RMSE/MAE, selecting the best model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

sns.set(style="whitegrid")
RANDOM_STATE = 42

# ---- 1. Load and prepare data ----
df = pd.read_csv("Walmart_Store_sales.csv")
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")  # file dates are in d-m-Y
df = df.sort_values(['Store', 'Date']).reset_index(drop=True)

# Additional time columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.to_period('Q')
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# ---- 2. Basic statistics tasks ----

# 2.1 Which store has maximum sales (total across period)?
store_total = df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False)
store_with_max_sales = store_total.idxmax()
print("Store with maximum total sales:", store_with_max_sales)
print("Total sales for that store: ${:,.2f}".format(store_total.loc[store_with_max_sales]))

# 2.2 Which store has maximum standard deviation i.e., sales vary a lot.
store_std = df.groupby('Store')['Weekly_Sales'].std().sort_values(ascending=False)
store_with_max_std = store_std.idxmax()
store_mean = df.groupby('Store')['Weekly_Sales'].mean()
coef_var = (store_std / store_mean).sort_values(ascending=False)  # coefficient of variation = std/mean
store_with_max_cv = coef_var.idxmax()
print("\nStore with maximum standard deviation:", store_with_max_std)
print("Standard deviation: ${:,.2f}".format(store_std.loc[store_with_max_std]))
print("Store with maximum coefficient of variation (std/mean):".format(store_with_max_cv))
print("Coefficient of variation: {:.4f}".format(coef_var.loc[store_with_max_cv]))

# 2.3 Which store(s) has good quarterly growth rate in Q3 2012
# We'll calculate two kinds of growth:
#  - QoQ: (Q3_2012 - Q2_2012) / Q2_2012
#  - YoY: (Q3_2012 - Q3_2011) / Q3_2011 (year-over-year)
q_sales = df.copy()
q_sales['YearQuarter'] = q_sales['Date'].dt.to_period('Q')
q_agg = q_sales.groupby(['Store', 'YearQuarter'])['Weekly_Sales'].sum().unstack(fill_value=np.nan)

q3_2012 = q_agg.get('2012Q3')  # Period label format "YYYYQn"
q2_2012 = q_agg.get('2012Q2')
q3_2011 = q_agg.get('2011Q3')

# compute growth safely (avoid division by zero)
qoQ = ((q3_2012 - q2_2012) / q2_2012).replace([np.inf, -np.inf], np.nan)
yoY = ((q3_2012 - q3_2011) / q3_2011).replace([np.inf, -np.inf], np.nan)

# pick stores with positive growth (both metrics positive) and rank them:
growth_df = pd.DataFrame({
    'Q3_2012': q3_2012,
    'Q2_2012': q2_2012,
    'Q3_2011': q3_2011,
    'QoQ_growth_Q3_2012_vs_Q2_2012': qoQ,
    'YoY_growth_Q3_2012_vs_Q3_2011': yoY
}).sort_values('QoQ_growth_Q3_2012_vs_Q2_2012', ascending=False)

print("\nTop 10 stores by QoQ growth (Q3 2012 vs Q2 2012):")
print(growth_df[['Q3_2012','Q2_2012','QoQ_growth_Q3_2012_vs_Q2_2012']].head(10))

# stores with clearly positive QoQ growth:
stores_positive_qoq = growth_df[growth_df['QoQ_growth_Q3_2012_vs_Q2_2012'] > 0].index.tolist()
print("\nStores with positive QoQ growth into Q3 2012 (vs Q2 2012):", stores_positive_qoq)

# 2.4 Holidays with higher sales than mean non-holiday sales
# Define holiday dates mapping (from problem statement) and filter only dates present in data
holiday_map = {
    'Super Bowl': ['2010-02-12', '2011-02-11', '2012-02-10'],
    'Labour Day': ['2010-09-10', '2011-09-09', '2012-09-07'],
    'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23'],
    'Christmas': ['2010-12-31', '2011-12-30']  # 2012-12-28 is outside dataset end maybe
}
# normalize map to datetimes that exist in the dataset
holiday_rows = []
for hname, dates in holiday_map.items():
    for d in dates:
        try:
            dt = pd.to_datetime(d, format="%Y-%m-%d")
            if dt in set(df['Date']):
                holiday_rows.append((hname, dt))
        except Exception:
            continue

# mean sales for non-holiday weeks (all stores together)
non_holiday_mean = df[df['Holiday_Flag'] == 0]['Weekly_Sales'].mean()
print("\nMean weekly sales across all stores during non-holiday weeks: ${:,.2f}".format(non_holiday_mean))

# compute avg total sales across all stores for each holiday date
holiday_stats = []
for hname, dt in holiday_rows:
    # sum across all stores for that week
    total_sales = df[df['Date'] == dt]['Weekly_Sales'].sum()
    holiday_stats.append({'Holiday': hname, 'Date': dt, 'Total_Sales': total_sales})

holiday_stats_df = pd.DataFrame(holiday_stats)
holiday_stats_df['HigherThanNonHolidayMean'] = holiday_stats_df['Total_Sales'] > non_holiday_mean * df['Store'].nunique()
# Note: non_holiday_mean is per-store-week; to compare to total across all stores multiply by number of stores.
print("\nHoliday weeks: total sales across stores and whether they exceed non-holiday average (all stores combined):")
print(holiday_stats_df[['Holiday','Date','Total_Sales','HigherThanNonHolidayMean']])

# 2.5 Monthly and semester view of sales in units and insights
# Monthly totals (all stores combined)
monthly = df.groupby([df['Date'].dt.to_period('M')])['Weekly_Sales'].sum().rename('Total_Sales').to_frame()
monthly.index = monthly.index.to_timestamp()
print("\nMonthly totals (first 12 rows):")
print(monthly.head(12))

# Semester view: define H1 = Jan-Jun, H2 = Jul-Dec per year
df['Semester'] = np.where(df['Month'] <= 6, 'H1', 'H2')
semester_totals = df.groupby(['Year','Semester'])['Weekly_Sales'].sum().unstack().fillna(0)
print("\nSemester totals by year (H1 / H2):")
print(semester_totals)

# Plotting monthly and semester (save figures)
plt.figure(figsize=(12,5))
monthly.plot(kind='line', legend=False, title='Total monthly sales (all stores)')
plt.ylabel('Total Weekly Sales (sum across stores)')
plt.tight_layout()
plt.savefig("monthly_total_sales.png", dpi=150)
plt.close()

plt.figure(figsize=(8,4))
semester_totals.plot(kind='bar', title='Semester sales by Year (H1 vs H2)')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.savefig("semester_sales_by_year.png", dpi=150)
plt.close()

print("\nSaved plots: monthly_total_sales.png and semester_sales_by_year.png")

# ---- 3. Statistical Model for Store 1 ----
# We'll prepare data, create a day-index, basic features, and compare Linear Regression vs RandomForest.
store1 = df[df['Store'] == 1].copy().sort_values('Date').reset_index(drop=True)

# create a numeric date index: days since first date
start_date = store1['Date'].min()
store1['day_index'] = (store1['Date'] - start_date).dt.days

# features: day_index, CPI, Unemployment, Fuel_Price, Temperature, Holiday_Flag, Month (cyclical)
store1['month_sin'] = np.sin(2 * np.pi * store1['Month'] / 12)
store1['month_cos'] = np.cos(2 * np.pi * store1['Month'] / 12)

features = ['day_index', 'CPI', 'Unemployment', 'Fuel_Price', 'Temperature', 'Holiday_Flag', 'month_sin', 'month_cos']
X = store1[features].ffill().bfill()  # Use ffill() and bfill() directly
y = store1['Weekly_Sales']

# Time-aware train/test split: last 12 weeks as test
test_weeks = 12
X_train, X_test = X[:-test_weeks], X[-test_weeks:]
y_train, y_test = y[:-test_weeks], y[-test_weeks:]

# Model 1: Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)

# Model 2: Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print("\nStore 1 prediction results (last {} weeks as test):".format(test_weeks))
print("Linear Regression  RMSE: ${:,.2f}, MAE: ${:,.2f}".format(rmse_lr, mae_lr))
print("Random Forest     RMSE: ${:,.2f}, MAE: ${:,.2f}".format(rmse_rf, mae_rf))

# Choose best model
best_model_name = "LinearRegression" if rmse_lr < rmse_rf else "RandomForest"
print("Selected best model based on RMSE:", best_model_name)

# show linear regression coefficients (if chosen)
coef_df = pd.DataFrame({'feature': features, 'coef': lr.coef_}).sort_values('coef', key=lambda s: s.abs(), ascending=False)
print("\nLinear Regression coefficients (by absolute magnitude):")
print(coef_df)

# Feature importances from RF
fi = pd.DataFrame({'feature': features, 'importance': rf.feature_importances_}).sort_values('importance', ascending=False)
print("\nRandom Forest feature importances:")
print(fi)

# Save predictions to CSV for inspection
predictions = store1[['Date', 'Weekly_Sales']].copy().iloc[-test_weeks:].reset_index(drop=True)
predictions['LR_Pred'] = y_pred_lr
predictions['RF_Pred'] = y_pred_rf
predictions.to_csv("store1_predictions_last_{}weeks.csv".format(test_weeks), index=False)
print("\nSaved predictions to store1_predictions_last_{}weeks.csv".format(test_weeks))

# Plot predictions vs actual for store 1 (last test_weeks)
plt.figure(figsize=(10,5))
plt.plot(predictions['Date'], predictions['Weekly_Sales'], label='Actual', marker='o')
plt.plot(predictions['Date'], predictions['LR_Pred'], label='LinearRegression', marker='x')
plt.plot(predictions['Date'], predictions['RF_Pred'], label='RandomForest', marker='s')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Weekly Sales')
plt.title('Store 1: actual vs predicted (last {} weeks)'.format(test_weeks))
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig("store1_predictions_comparison.png", dpi=150)
plt.close()
print("Saved plot store1_predictions_comparison.png")

# ---- 4. Quick notes / insights (automated suggestions) ----
insights = """
Automated insights summary:
- Store with maximum total sales: {} (see store_total series).
- Store with maximum variability (std): {}.
- Several stores showed positive QoQ growth into Q3 2012 (list printed).
- Holiday weeks comparison: see holiday_stats_df printed above; those holiday weeks with totals above
  the aggregate non-holiday mean (scaled to number of stores) are flagged.
- Monthly and semester aggregates saved as PNGs. Inspect monthly_total_sales.png for seasonality,
  and semester_sales_by_year.png for H1/H2 comparisons.
- Modeling (Store 1): compared a temporal linear model vs RandomForest (features: day index, CPI,
  Unemployment, Fuel_Price, Temperature, Holiday_Flag, cyclical month). The script reports RMSE/MAE.
  RandomForest often captures non-linearity and interactions; LinearRegression is interpretable (coeffs printed).
""".format(store_with_max_sales, store_with_max_std)

with open("insights_summary.txt","w") as f:
    f.write(insights)

print("\nWrote automated insights to insights_summary.txt")

# ---- 5. Recommendations & next steps ----
recommendations = """
Recommended next steps (for better accuracy and business value):
1) Add lag features and rolling means (e.g., sales lag-1, lag-2, 4-week rolling mean) to capture temporal autocorrelation.
2) Use time-series-specific models: SARIMAX (with exogenous variables), FB Prophet (now called 'prophet'),
   or gradient boosting (XGBoost/LightGBM) with time features + lags.
3) Engineer markdown/promotion features: the dataset lacks explicit markdown amounts — try to infer markdown events
   from sudden sales uplift patterns or introduce retailer markup calendars if available.
4) Cross-validate using rolling (time-series) CV instead of a single holdout; tune hyperparameters for tree-based models.
5) Evaluate business metric: weight holiday-week errors more (per problem statement) — e.g., multiply errors on
   the four major holiday weeks by 5 to reflect competition scoring.
6) For supply chain decisions, produce prediction intervals (quantiles) instead of point estimates.
"""

with open("next_steps.txt","w") as f:
    f.write(recommendations)

print("\nWrote recommended next steps to next_steps.txt")
'''

    # Replace problematic character for latin-1 encoding for code content as well.
    code_content = code_content.replace('—', '--')
    pdf.multi_cell(0, 4, code_content)
    pdf.ln(10)

    pdf.output(output_filename)
    print(f"Report saved as {output_filename}")

create_report_pdf()

Report saved as retail_analysis_report.pdf
